In [ ]:
import pandas as pd
import geopandas as gpd
import hvplot.pandas
# import cartopy.crs as ccrs

# load csv file

In [ ]:
df = pd.read_csv('data-flooddamage/data.csv',index_col=0)

# make bar chart

In [ ]:
count = df['date'].value_counts(sort=False)
scount = count.sort_index()

d = pd.DataFrame(scount)
d.columns = ['Damaged houses']
d.index.name = 'date'

g = d.hvplot.bar(y='Damaged houses', rot=30, width=400, height=250)  # + d.hvplot.table(['date', 'Damaged houses'], index_position=None, width=200)

# make scatter plot

In [ ]:
df2 = df[df['longitude'] != 0]
gdf = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.longitude, df2.latitude))
gdf = gdf.drop(['longitude','latitude'],axis = 1)
# gdf['date'] = pd.to_datetime(gdf['date'])
gdf.crs = 'epsg:4326'

In [ ]:
from bokeh.models import HoverTool
hover = HoverTool(tooltips=[ ("address", "@address"),("damage", "@damage"),("cause", "@cause")])

In [ ]:
g2 = gdf.hvplot.points(geo=True, groupby='date', frame_height=400, frame_width=500, tiles=False, size=100, color='red'
#                   ,crs = ccrs.PlateCarree(), project=True, projection=ccrs.PlateCarree()
                       , hover_cols = ['address','damage','cause']
                      ).options(tools=[hover])

# make polygon

In [ ]:
poly = gpd.read_file('chofu.geojson')
p = poly.hvplot.polygons( geo=True, frame_height=400, frame_width=500, tiles=True, line_color='black', line_width=5, alpha=0.2, hover=False )

# layout

In [ ]:
layout = ( g + p*g2).cols(2)

# on bokeh server

In [ ]:
import holoviews as hv

renderer = hv.renderer('bokeh')

In [ ]:
doc = renderer.server_doc(layout)
doc.title = 'chofu flood damage'